# 6. 装饰器（注解）

## 6.1 属性的懒加载
问题：如何让一个属性在第一次使用时才计算，后面缓存起来
解决：使用一个描述器类来实现

In [ ]:
class lazyproperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
        setattr(instance, self.func.__name__, value)
        return value

import math

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius

c = Circle(4.0)
print(vars(c))

print(c.area)
# 查看对象c，发现c.area已经有值了
print(vars(c))
# 第二次调用 c.area，只会输出一次"Computing area"，说明并没有真正地
print(c.area)

# 删除属性
del c.area
print(vars(c))


- 当一个描述器被放入一个类的定义时，每次访问属性时它的 __ get __()、 __ set __() 和 __ delete __() 方法就会被触发
- 如果一个描述器仅仅只定义了一个 __ get __() 方法的话，它比通常的具有更弱的绑定：只有当被访问属性不在实例底层的字典中时 __ get __() 方法才会被触发


## 6.2 在函数上添加包装器

问题：函数上添加一个包装器，增加额外的操作处理 (比如日志、计时等)
解决：使用装饰器函数

In [1]:
import time
from functools import wraps

def timethis(func):
    '''
    用于输出程序执行时间的装饰器
    '''

    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

countdown(100000)
countdown(10000000)

countdown 0.007214069366455078


以下两种写法是等价的：

In [ ]:
# 方式一
@timethis
def countdown(n):
    pass

In [ ]:
# 方式二
def countdown(n):
    pass
countdown = timethis(countdown)


## 6.3 创建装饰器时保留函数元信息
问题：写了一个装饰器作用在某个函数上，怎样使这个函数的元信息比如名字、文档字符串、注解和参数签名不丢失
解决：任何时候定义装饰器时，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数

In [4]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    
    # 必须要用wraps进行包装
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n:int):
    """
    Counts down
    """
    
    while n > 0:
        n -= 1

countdown(10000)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)

countdown 0.0008609294891357422
countdown

    Counts down
    
{'n': <class 'int'>}


## 6.4 解除一个装饰器
问题：一个装饰器已经作用在一个函数上，想撤销它，直接访问原始的未包装的那个函数
解决：通过访问 __ wrapped __ 属性来访问原始函数
注意：如果有多个包装器，那么访问 wrapped 属性的行为是不可预知的，应该避免这样做

In [11]:
import time
from functools import wraps

def timethis(func):
    """
    Decorator that reports the execution time.
    """
    
    # 必须要用wraps进行包装
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def add(x, y):
    return x+y

print(add(10, 5))

print("\n解除装饰器：")
orig_add = add.__wrapped__
print(orig_add(10, 5))

add 2.1457672119140625e-06
15

解除装饰器：
15


## 定义一个带参数的装饰器
问题：怎样定义一个可以接受参数的装饰器
解决：最外层的函数 logged() 接受参数并将它们作用在内部的装饰器函数上面。内层的函数 decorate() 接受一个函数作为参数，然后在函数上面放置一个包装器。这里的关键点是包装器是可以使用传递给 logged() 的参数的。

In [ ]:
from functools import wraps
import logging
def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')
